In [7]:
from pyspark import *
from pyspark.sql import SparkSession

import os
import socket

pod_ip = socket.gethostbyname(socket.gethostname())
os.environ['PYSPARK_PYTHON'] = 'python3' # Needs to be explicitly provided as env. Otherwise workers run Python 2.7
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3'  # Same

conf = SparkConf([('spark.executor.instances', '5'),
            ('spark.driver.memory', '4g'),
            ('spark.executor.memory', '16g'),
            ('spark.driver.host', pod_ip),
            ("spark.sql.sources.partitionOverwriteMode","dynamic"),
            ('spark.driver.extraJavaOptions', '-Dio.netty.tryReflectionSetAccessible=true'),
            ('spark.executor.extraJavaOptions', '-Dio.netty.tryReflectionSetAccessible=true'),
            ('spark.kubernetes.container.image', 'mccho8865/spark-py:3.0.2'),
            ('spark.master', 'k8s://https://kubernetes.default.svc'),
            ('spark.submit.deployMode', 'client'),
            ('spark.hadoop.fs.s3a.fast.upload', 'true'),
            ('spark.hadoop.fs.s3a.endpoint', 'rook-ceph-rgw-my-store.rook-ceph.svc'),
            ('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'),
            ('spark.hadoop.fs.s3a.path.style.access', 'true'),
            ('spark.hadoop.fs.s3a.access.key', 'Z780FG2AP64YD0Y2EWS8'),
            ('spark.hadoop.fs.s3a.secret.key', 'akGdNm3vY9xSCcyscq8StdTh6BMRGtt9FChidPgn'),
            ('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem'),
            ('spark.kubernetes.file.upload.path', 's3a://spark'),
            ('spark.kubernetes.node.selector.spark'),
            ('spark.sql.session.timeZone', 'Asia/Seoul'),
            ('spark.driver.extraJavaOptions', '-Duser.timezone=Asia/Seoul'),
            ('spark.executor.extraJavaOptions', '-Duser.timezone=Asia/Seoul'),
            ('spark.eventLog.enabled', 'True'),
            ('spark.eventLog.dir', 's3a://spark-hs//')
            ])
# conf.setAppName('rawfile_to_parquet')


sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)

Exception: Java gateway process exited before sending its port number

In [ ]:
dt = 

In [2]:
raw_df = spark.read.option("mode", "DROPMALFORMED").text('s3a://coin-bucket/warehouse/raw/ticker/')
raw_df = raw_df.where("dt >= 20210607")
# raw_df = raw_df.where("20210513 <= dt and dt < 20210610")

In [3]:
raw_df.explain()

== Physical Plan ==
FileScan text [value#0,dt#1] Batched: false, DataFilters: [], Format: Text, Location: InMemoryFileIndex[s3a://coin-bucket/warehouse/raw/ticker], PartitionFilters: [isnotnull(dt#1), (dt#1 >= 20210607)], PushedFilters: [], ReadSchema: struct<value:string>




In [4]:
raw_df.show()

+--------------------+--------+
|               value|      dt|
+--------------------+--------+
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
|{    "data": {   ...|20210829|
+--------------------+--------+
only showing top 20 rows



In [5]:
raw_df.explain()

== Physical Plan ==
FileScan text [value#0,dt#1] Batched: false, DataFilters: [], Format: Text, Location: InMemoryFileIndex[s3a://coin-bucket/warehouse/raw/ticker], PartitionFilters: [isnotnull(dt#1), (dt#1 >= 20210607)], PushedFilters: [], ReadSchema: struct<value:string>




In [6]:
raw_df.groupby('dt').count().show()

+--------+-----+
|      dt|count|
+--------+-----+
|20210910|86399|
|20210827|86380|
|20210911|86400|
|20210901|86388|
|20210908|86400|
|20210907|86400|
|20210830|86400|
|20210903|86400|
|20210905|86400|
|20210902|86380|
|20210912|86400|
|20210829|86400|
|20210913|86393|
|20210828|86400|
|20210904|86400|
|20210906|86398|
|20210914|86400|
|20210831|86400|
|20210909|86400|
+--------+-----+



In [7]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *
value_schema = StructType([ StructField('data', StringType()),
                            StructField('status', StringType())])
value_df = raw_df.withColumn('value_json', from_json('value', value_schema)) \
                 .select(col('value_json.data').alias('data'), col('value_json.status').alias('status'))
value_df = value_df.filter('status = 0000').select('data')
value_df.cache()
# value_df.limit(10).toPandas()['data'][0]

DataFrame[data: string]

In [8]:
value_df.show()

+--------------------+
|                data|
+--------------------+
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
|{"AAVE":{"acc_tra...|
+--------------------+
only showing top 20 rows



In [9]:
ticker_schema = StructType([StructField('acc_trade_value', StringType(), True),
                            StructField('acc_trade_value_24H', StringType(), True),
                            StructField('closing_price', StringType(), True),
                            StructField('fluctate_24H', StringType(), True),
                            StructField('fluctate_rate_24H', StringType(), True),
                            StructField('max_price', StringType(), True),
                            StructField('min_price', StringType(), True),
                            StructField('opening_price', StringType(), True),
                            StructField('prev_closing_price', StringType(), True),
                            StructField('units_traded', StringType(), True),
                            StructField('units_traded_24H', StringType(), True),
                            StructField('coin', StringType(), True),
                            StructField('timestamp', StringType(), True)])
ticker_array_schema = ArrayType(ticker_schema, True)

In [10]:
import json
from pyspark.sql.functions import udf, explode, from_unixtime, to_date
@udf(ticker_array_schema)
def parse_raw_ticker(data):
    data = json.loads(data)
    timestamp = data['date']
    del data['date']
    coins = list(data.keys())
    out = []
    for coin in coins:
        coin_item = data[coin]
        coin_item["coin"] = coin
        coin_item["timestamp"] = timestamp
        out.append(coin_item)
    return out

In [11]:
parsed_df = value_df.withColumn('parsed_arr', explode(parse_raw_ticker('data')))
parsed_df = parsed_df.select('parsed_arr.*')

In [12]:
parsed_df.show(10)

+----------------+-------------------+-------------+------------+-----------------+---------+---------+-------------+------------------+------------------+------------------+-----+-------------+
| acc_trade_value|acc_trade_value_24H|closing_price|fluctate_24H|fluctate_rate_24H|max_price|min_price|opening_price|prev_closing_price|      units_traded|  units_traded_24H| coin|    timestamp|
+----------------+-------------------+-------------+------------+-----------------+---------+---------+-------------+------------------+------------------+------------------+-----+-------------+
|  311026018.1799|     311026018.1799|       426800|      -11800|            -2.69|   442000|   424700|       438600|            438300|      715.63965039|      715.63965039| AAVE|1630162799073|
|75165933158.4822|   75174324180.0117|         3291|         -66|            -1.97|     3460|     3248|         3361|              3360| 22412845.91658426| 22415343.84443834|  ADA|1630162799073|
| 2090393985.3257|    209

In [13]:
out_df = parsed_df.select(col('coin').cast(StringType()),
                         from_unixtime(col('timestamp')/1000).alias('timestamp'),
                         col('opening_price').cast(DoubleType()),
                         col('closing_price').cast(DoubleType()),
                         col('min_price').cast(DoubleType()),
                         col('max_price').cast(DoubleType()),
                         col('units_traded').cast(DoubleType()),
                         col('acc_trade_value').cast(DoubleType()),
                         col('prev_closing_price').cast(DoubleType()),
                         col('units_traded_24H').cast(DoubleType()),
                         col('acc_trade_value_24H').cast(DoubleType()),
                         col('fluctate_24H').cast(DoubleType()),
                         col('fluctate_rate_24H').cast(DoubleType()),
                         from_unixtime(col('timestamp')/1000, 'yyyy-MM-dd').alias('dt'))
out_df = out_df.distinct()

In [14]:
out_df.repartition('coin', 'dt').write.partitionBy(['coin', 'dt']).mode('append').parquet('s3a://coin-bucket/warehouse/data/ticker')

In [15]:
spark.stop()